In [148]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV, LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [149]:
df = pd.read_csv("data.csv")

In [150]:
target = df["D"].shift(-1)
target = target.dropna()

In [151]:
df = df[:-1]

In [152]:
for index, columns in enumerate(["A", "B", "C", "D"]):
    print(i, columns)
    for i in range(len(df)):
        fill_value = df[columns].iloc[i]
        df[f"{columns}_Lag_{i}"] = df[columns].shift(-i, fill_value=fill_value)


3 A
226 B
226 C
226 

C:\Users\Skwix\AppData\Local\Temp/ipykernel_4004/873558226.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{columns}_Lag_{i}"] = df[columns].shift(-i, fill_value=fill_value)


D


In [153]:
df.drop(["A","B","C","D"],axis=1,inplace=True)

In [154]:
df.isnull().sum().sum()

0

In [155]:
target.isnull().sum().sum()

0

In [156]:
target

0      0.7
1      0.9
2      1.1
3      0.9
4      0.9
      ... 
222    1.5
223    1.6
224    1.4
225    1.5
226    1.3
Name: D, Length: 227, dtype: float64

In [157]:
x_train, x_test, y_train, y_test = train_test_split(df,target)

# Standard Linear Regression

In [158]:
model = LinearRegression()

In [159]:
model.fit(x_train,y_train)

LinearRegression()

In [160]:
x_test.head(1)

,A_Lag_0,A_Lag_1,A_Lag_2,A_Lag_3,A_Lag_4,A_Lag_5,A_Lag_6,A_Lag_7,A_Lag_8,A_Lag_9,...,D_Lag_217,D_Lag_218,D_Lag_219,D_Lag_220,D_Lag_221,D_Lag_222,D_Lag_223,D_Lag_224,D_Lag_225,D_Lag_226
117,7.85,8.09,8.36,8.75,9.06,9.36,9.52,9.62,9.74,9.86,...,2.0,1.5,1.9,1.4,1.3,1.3,1.5,1.6,1.4,1.5


In [161]:
model.predict(x_test)

array([ 2.35481677,  1.39286735,  2.67660816,  2.31107878,  1.63086166,
        2.59132257,  0.35787694,  1.49498141,  2.30464589,  0.2854191 ,
        0.85704721,  2.49556378,  2.3179988 ,  2.30018993,  1.05099502,
        3.55596256,  0.82983638,  0.27831789,  2.32005582,  2.23478985,
        2.06929428,  1.23142135,  2.15222245,  2.38169825,  2.27620676,
        1.45942335,  0.44531425, -0.13877928,  0.92474743,  0.77455911,
        3.36489825,  1.79331587,  2.17668038,  2.81144793,  2.71341244,
        2.08831773,  1.96102033,  0.03913256,  2.29748436,  1.11987385,
        1.87608212,  1.86258745,  3.407668  ,  2.29080788,  2.40116598,
        1.29839549,  1.78708739,  3.32967145,  1.36178438,  2.32186202,
        2.22102734,  1.90310746,  1.90094926,  1.90306067,  2.74328889,
        2.73179562,  2.04353172])

In [162]:
r2_score(y_test, model.predict(x_test))

0.9662815771088938

# Regularisation

In [163]:
myalphas = np.logspace(-6, 6, 13)
train_sizes = 150, 10

models = LinearRegression, RidgeCV, LassoCV
model_names = 'linear regression', 'Ridge regression using CV', 'Lasso regression using CV'

for train_size in train_sizes:
    print('Training with this many datapoints: ', train_size)
    for i, model in enumerate(models):
        print('Using regression of type: ', model_names[i])
        if model == LinearRegression:
            reg = model()
        else:
            print('Trying these complexity parameter values (i.e.alphas): ', myalphas)
            reg = model(alphas=myalphas)
        reg.fit(x_train,y_train)

        if model != LinearRegression:
            print('Using cross-validation got this value for the complexity parameter: ', reg.alpha_)
        for i, name in enumerate(["A","B","C","D"]):
            print('Parameter for {0} is {1}'.format(name,reg.coef_[i]))

        print('Error on held out set is: ', reg.score(x_test,y_test))
        print('\n\n')

Training with this many datapoints:  150
Using regression of type:  linear regression
Parameter for A is -0.014456340492597261
Parameter for B is -0.022742167710996644
Parameter for C is -0.005885970674734085
Parameter for D is -0.03841114559302499
Error on held out set is:  0.9662815771088938



Using regression of type:  Ridge regression using CV
Trying these complexity parameter values (i.e.alphas):  [1.e-06 1.e-05 1.e-04 1.e-03 1.e-02 1.e-01 1.e+00 1.e+01 1.e+02 1.e+03
 1.e+04 1.e+05 1.e+06]
Using cross-validation got this value for the complexity parameter:  1e-06
Parameter for A is -0.014335557329071186
Parameter for B is -0.02298488817181485
Parameter for C is -0.006088181675453941
Parameter for D is -0.03847021748954685
Error on held out set is:  0.966249990010038



Using regression of type:  Lasso regression using CV
Trying these complexity parameter values (i.e.alphas):  [1.e-06 1.e-05 1.e-04 1.e-03 1.e-02 1.e-01 1.e+00 1.e+01 1.e+02 1.e+03
 1.e+04 1.e+05 1.e+06]


C:\Users\Skwix\anaconda3\envs\int\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.855e-02, tolerance: 1.566e-02
  model = cd_fast.enet_coordinate_descent(


Using cross-validation got this value for the complexity parameter:  1e-05
Parameter for A is -0.4405188487261763
Parameter for B is -0.003164872781015995
Parameter for C is 0.0005010401786213362
Parameter for D is -0.021371940752024716
Error on held out set is:  0.963831337542275



Training with this many datapoints:  10
Using regression of type:  linear regression
Parameter for A is -0.014456340492597261
Parameter for B is -0.022742167710996644
Parameter for C is -0.005885970674734085
Parameter for D is -0.03841114559302499
Error on held out set is:  0.9662815771088938



Using regression of type:  Ridge regression using CV
Trying these complexity parameter values (i.e.alphas):  [1.e-06 1.e-05 1.e-04 1.e-03 1.e-02 1.e-01 1.e+00 1.e+01 1.e+02 1.e+03
 1.e+04 1.e+05 1.e+06]
Using cross-validation got this value for the complexity parameter:  1e-06
Parameter for A is -0.014335557329071186
Parameter for B is -0.02298488817181485
Parameter for C is -0.006088181675453941
Parameter for D is

C:\Users\Skwix\anaconda3\envs\int\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.855e-02, tolerance: 1.566e-02
  model = cd_fast.enet_coordinate_descent(
